# Feature Enginnering

### Objectives:- 
1. Construction
2. 

## Feature Construction
##### It invloves creating new or modified features that help ML models perform better

In [13]:
import pandas as pd
#eg - Creating derived features
data = {
    "Name":["A","B","C","D"],
    "Python":[10,20,30,40],
    "MLC":[1,2,3,4]
}
df = pd.DataFrame(data)
df["Final Mark"]=df["Python"]+ 2*(df["MLC"])
df

,Name,Python,MLC,Final Mark
0,A,10,1,12
1,B,20,2,24
2,C,30,3,36
3,D,40,4,48


In [3]:
#Encoding Normal Variables
data = {
    'city' :  ['A',"B",'C','A'],
    'parents_athlete': ['Y','N','Y','N'],
    'chance_of_win':['Y','N','Y',"N"]
}
df = pd.DataFrame(data)
print(df)
# pd.get_dummies(df['city'])
pd.get_dummies(df,drop_first=False)

  city parents_athlete chance_of_win
0    A               Y             Y
1    B               N             N
2    C               Y             Y
3    A               N             N


,city_A,city_B,city_C,parents_athlete_N,parents_athlete_Y,chance_of_win_N,chance_of_win_Y
0,1,0,0,0,1,0,1
1,0,1,0,1,0,1,0
2,0,0,1,0,1,0,1
3,1,0,0,1,0,1,0


#### Explanation:-

###### get_dummies() converts each catergorical data to binary data

In [4]:
# Name , city , blood group , cgpa , which we need in binary ? 
data = {
    "Name":['A',"B","C",'D'],
    "City":['C1',"C2","C1","C3"],
    "Blood_Group":['A',"B","O","B"],
    "CGPA":[8,9,10,10]
}
df = pd.DataFrame(data)
print(df)
dff = pd.get_dummies(df, columns =['City','Blood_Group'])
dff

  Name City Blood_Group  CGPA
0    A   C1           A     8
1    B   C2           B     9
2    C   C1           O    10
3    D   C3           B    10


,Name,CGPA,City_C1,City_C2,City_C3,Blood_Group_A,Blood_Group_B,Blood_Group_O
0,A,8,1,0,0,1,0,0
1,B,9,0,1,0,0,1,0
2,C,10,1,0,0,0,0,1
3,D,10,0,0,1,0,1,0


In [5]:
#Encoding ordinal vars
data = {"grade":['A','B',"C","D","A"]}
df=pd.DataFrame(data)
grade_map = {'A':1,"B":2,"C":3,"D":4}
df['num_grade']=df['grade'].map(grade_map)
df

,grade,num_grade
0,A,1
1,B,2
2,C,3
3,D,4
4,A,1


#### Explanation:-
##### Uses .map() to apply to grade_map to the 'grade' col .Creates a new col 'num_grade' with the corresponding numerical values

In [6]:
# Binning Continues Variables
import numpy as np
df = pd.DataFrame({'price':[20000,35000,60000,80000]})
bins = [0,30000,60000,np.inf]
labels = ['low','medium','high']
df['price_category']=pd.cut(df['price'],bins=bins,labels=labels)
df

,price,price_category
0,20000,low
1,35000,medium
2,60000,medium
3,80000,high


#### Explanation:-
##### bins defines the edges of the price ranges:
* 0-30000 -> Low
* 30000-60000 -> Medium
* 60000 + -> High

## Feature Extraction
##### It reduces the number of features while preserving important info

In [23]:
#PCA example
from sklearn.decomposition import PCA
from sklearn.datasets import load_iris

#LOAD iris dataset
iris = load_iris()
X = iris.data
# print(X)
# print(iris)

#apply PCA(reduce to 2 components)
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X)

print("Original Shape: ",X.shape)
print("Reduced Shape: ",X_pca.shape)

Original Shape:  (150, 4)
Reduced Shape:  (150, 2)


In [8]:
#SVD Example
import numpy as np
from sklearn.decomposition import TruncatedSVD

#Random matrix(simulating text data)
X=np.random.rand(5,4)

#Apply Truncated SVD
svd=TruncatedSVD(n_components=2)
X_svd = svd.fit_transform(X)

print("Original: ",X.shape)
print("Reduced: ",X_svd.shape)

Original:  (5, 4)
Reduced:  (5, 2)


In [9]:
#LDA Example
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.datasets import load_iris

iris = load_iris()
X,y = iris.data, iris.target

lda = LinearDiscriminantAnalysis(n_components=2)
X_lda=lda.fit_transform(X,y)

print("Original: ",X.shape)
print("Reduced: ",X_lda.shape)

Original:  (150, 4)
Reduced:  (150, 2)


## Feature Selection
##### It identifies the most relevant features

In [10]:
#Filter Method(Chi-Square)
from sklearn.feature_selection import SelectKBest, chi2
X_new = SelectKBest(chi2,k=2).fit_transform(iris.data,iris.target)
print("Original: ",iris.data.shape,'Reduced: ',X_new.shape)

Original:  (150, 4) Reduced:  (150, 2)


In [12]:
#Alternate Method(Chi-Square)
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.datasets import load_iris

#Load iris dataset as DataFrame
iris = load_iris(as_frame=True)
df_i = iris.frame

#Separate features and target using iloc
X= df_i.iloc[:,:-1] #all columns except last
y = df_i.iloc[:,-1] #last column

#Apply Chi-Squre test
X_new = SelectKBest(chi2,k=1).fit_transform(X,y)

print("Original shape: ",X.shape)
print("Reduced shape: ",X_new.shape)

Original shape:  (150, 4)
Reduced shape:  (150, 1)


#### Explanation:-
.iloc ensures we slect the independent vars(X) and dependent vars(y) directly by index pos, regardless of col names

In [18]:
#Wrapper Method- Recurrsive Feature Elimination(RFE)
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
X,y=iris.data,iris.target
model = LogisticRegression(max_iter=200)

#Apply RFE for 2 features
rfe = RFE(model,n_features_to_select=2)
fit = rfe.fit(X,y)

print("Selected Features: ",fit.support_)
print("Feature Ranking: ",fit.ranking_)

Selected Features:  [False False  True  True]
Feature Ranking:  [3 2 1 1]


In [24]:
#Embedded Method (Lasso)
from sklearn.linear_model import LassoCV
from sklearn.datasets import load_diabetes

#Load diabetes dataset as DataFrame
diabetes = load_diabetes(as_frame=True)
df_d = diabetes.frame

#Separate features and target using iloc
X= df_d.iloc[:,:-1]
y = df_d.iloc[:,-1]

#Apply LassoCV
lasso = LassoCV(cv=5)
lasso.fit(X,y)

print("Coefficients: ",lasso.coef_)
print("No of selected features: ",sum(lasso.coef_!=0))

Coefficients:  [  -6.49469328 -235.99308032  521.7443693   321.0607768  -569.43813385
  302.45319289   -0.          143.69851474  669.92267515   66.83551067]
No of selected features:  9
